In [ ]:
import httpx

from loguru import logger


class Downloader(httpx.Client):
    def __init__(
        self,
        url: str = None,
        api_key: str = None,
    ) -> None:
        super(Downloader, self).__init__(
            base_url=url,
            headers={
                "Content-Type": "application/json",
            },
            timeout=30,
        )

        self.query_params = {"apiKey": api_key}

    def get_all_audio_files(self):
        response: httpx.Response = self.get(f"/api/getMp3s", params=self.query_params)

        if response.status_code != 200:
            raise Exception(response.json())

        return response.json()

    def download_file(self, link: str = None):
        body = {
            "url": link,
            "type": "audio",
            "cropFileSettings": {"cropFileStart": 0, "cropFileEnd": 0},
        }

        response: httpx.Response = self.post(
            f"/api/downloadFile", json=body, params=self.query_params
        )

        if response.status_code != 200:
            raise Exception(response.json())

        return True

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from youtubesearchpython import VideosSearch

# Get HTML using Webdriver with BeautifulSoup
driver = webdriver.Chrome(ChromeDriverManager().install())
url = "https://www.melon.com/chart/index.htm"
driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# Make a DataFrame
titles = [song.text.strip("\n") for song in soup.select("div.ellipsis.rank01")]
singers = [
    singer.text for singer in soup.select("div.ellipsis.rank02 > span.checkEllipsis")
]
logger.info(f"singers: {len(singers)}")

# Close webdriver
driver.close()
driver.quit()

downloader = Downloader(
    url="http://localhost:8998", api_key="9a49884a-ffe7-4d4e-8ccd-6a313e31c760"
)
for singer, title in zip(singers, titles):
    search_results = VideosSearch(f"{singer} - {title} Lyrics", limit=1).result()[
        "result"
    ]

    title = search_results[0]["title"]
    link = search_results[0]["link"]

    print(f"title: {title} link: {link}")
    downloader.download_file(link=link)

print("Download Done!!")

In [ ]:
# http://localhost:13378/api/libraries/lib_m7ek971o4xw0sksn3b/scan?force=0
class Player(httpx.Client):
    def __init__(
        self,
        url: str = None,
        username: str = "admin",
        password: str = "admin",
    ) -> None:
        body: dict = {"username": username, "password": password}

        response: httpx.Response = httpx.post(
            f"{url}/login",
            json=body,
        )

        # retry login
        if response.status_code != 200:
            raise ValueError("Bad credentials")

        print(response.json()["user"]["token"])
        super(Player, self).__init__(
            base_url=url,
            headers={
                "Content-Type": "application/json",
                "Authorization": f"""Token {response.json()['user']['token']}""",
            },
            timeout=30,
        )

    def get_all_libraries(self):
        response: httpx.Response = self.get(f"/api/libraries")

        if response.status_code != 200:
            raise Exception(response.content)

        return response.json()

    def get_item_in_libraries(self, lib_id: str = None):
        response: httpx.Response = self.get(f"/api/items/{lib_id}?expanded=1")

        if response.status_code != 200:
            raise Exception(response.content)

        return response.json()

    def match_all_library_items(self, lib_id: str = None):
        response: httpx.Response = self.get(f"/api/libraries/{lib_id}/matchall")

        if response.status_code != 200:
            raise Exception(response.content)

        print(response.content)
        # return response.json()

    def scan_library(self, lib_id: str = None):
        response: httpx.Response = self.post(f"/api/libraries/{lib_id}/scan")

        if response.status_code != 200:
            raise Exception(response.content)

        print(response.content)


player = Player(url="http://localhost:13378")
lib_list = player.get_all_libraries()
lib_id = lib_list["libraries"][0]["id"]
print("lib_id: ", lib_id)
print(lib_list["libraries"][0])

player.scan_library(lib_id=lib_id)